<a href="https://colab.research.google.com/github/hawc2/contentdm-iiif-api/blob/main/IIIF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal

First goal: Download the images with metadata from ContentDM or OAI-PMH

Second goal: Feed images into the facial recognition pipeline

# Source 

This is adapting this Smithsonian notebook https://github.com/hibernator11/notebook-iiif-images


# Questions

How to connect IIIF API with other ContentDM or OAI-PMH endpoints?

https://iiif.io/api/cookbook/

# Issues

find "thumbnails" in ContentDM IIIF API. 

# To Do

Create variable for user to set which iiif collection to download

# API Calls


/https://digital.library.temple.edu/iiif/{collectionID}/{objectID}/info.json

https://digital.library.temple.edu/iiif/info/p16002coll4/manifest.json

 /iiif/{collectionID}/{objectID}/{region}/{size}/{rotation}/{quality}.{format}

# Temple API Info

Manifest: https://digital.library.temple.edu/iiif/info/p16002coll4/manifest.json

Using this digital collection item: https://digital.library.temple.edu/digital/collection/p16002coll9/id/2923/rec/1

Temple's Image API example: 
https://digital.library.temple.edu/digital/iiif/p16002coll9/2923/full/pct:15/0/default.jpg

Relevant endpoint: https://digital.library.temple.edu/oai/oai.php



## ContentDM API Test:

https://platform.worldcat.org/api-explorer/apis/CONTENTdmIIIFImageAPI/Image/GetInformation

https://platform.worldcat.org/api-explorer/apis/CONTENTdmIIIFImageAPI/Image/GetImage


# Install Packages


In [12]:
import requests, csv
import json
import pandas as pd
import cv2
import os
import urllib.request
import urllib3
from urllib3 import request
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from random import randint
import ipywidgets as widgets
from IPython.display import Image, display
from IPython.core.display import HTML 
from PIL import Image, ImageDraw, ImageFont, ImageOps
from io import BytesIO
import certifi

# Query ContentDM

## Query a Collection

The manifestURL variable determines what collection, such as WWII Posters, are queried. 

In [13]:
#manifestUrl = 'https://cdm16002.contentdm.oclc.org/iiif/info/manifest.json'
manifestUrl = 'https://cdm16002.contentdm.oclc.org/iiif/info/p16002coll9/p1.json'

In [14]:
responseManifest = requests.get(manifestUrl)
print(responseManifest.url)

# retrieving the metadata
m = json.loads(responseManifest.text)

#for i in m['manifests']:
#    print(i['@id'])

#print(m)

https://cdm16002.contentdm.oclc.org/iiif/info/p16002coll9/p1.json


In [16]:
# Save Metadata as CSV

csv_out = csv.writer(open('tul.csv', 'w'), delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
csv_out.writerow(['Title', 'Date', 'Contributor', 'Description', 'Subject'])

44

In [15]:
for i in m['manifests']:
    title = date = contributor = description = subject = ''
    
    manifestItemUrl = i['@id']
    responseManifestItem = requests.get(manifestItemUrl)
    manifestItem = json.loads(responseManifestItem.text)

    title = manifestItem['metadata'][0]['value']
    date = manifestItem['metadata'][1]['value']
    contributor = manifestItem['metadata'][2]['value']
    description = manifestItem['metadata'][3]['value']
    subject = manifestItem['metadata'][4]['value']
    #attribution = manifestItem['attribution']

    #print(title + " " + date)
    csv_out.writerow([title, date, contributor, description, subject])    

NameError: ignored

## Broken Code Chunk

In [ ]:
pid = 0
img_urls = {}

for i in m['manifests']:
    manifestItemUrl = i['@id']
    responseManifestItem = requests.get(manifestItemUrl)
    manifestItem = json.loads(responseManifestItem.text)
    for c in data['sequences'][0]['canvases']:
        img_urls[pid]=c['images'][0]['resource']['service']['@id']
        print (img_urls[pid])
        pid = pid +1
print ("Ready.")

In [ ]:
from google.colab import files

files.download('tul.csv')

# Exploring

In [ ]:
df

In [ ]:
# Get unique values
artist = pd.unique(df['Title'].str.split('|', expand=True).stack()).tolist()
for a in sorted(artist):
    print(a)

In [ ]:
# Splits the people column and counts frequencies
artist_counts = df['contributor'].str.split('|').apply(lambda x: pd.Series(x).value_counts()).sum().astype('int').sort_values(ascending=False).to_frame().reset_index(level=0)
# Add column names
artist_counts.columns = ['Contributor', 'count']
# Display with horizontal bars
display(artist_counts.style.bar(subset=['count'], color='#d65f5f').set_properties(subset=['count'], **{'width': '300px'}))

In [ ]:
# Get unique values
types = pd.unique(df['type']).tolist()
for type in sorted(types, key=str.lower):
    print(type)

# Everything Below Under Revision

## Alternate Query (not working)

This might work with the general temple manifest

In [ ]:
#https://digital.library.temple.edu/iiif/info/p16002coll4/manifest.json

#https://cdm16002.contentdm.oclc.org/iiif/image/' + p16002coll4 +'/full/full/0/default.jpg
#https://cdm16002.contentdm.oclc.org/iiif/image/p16002coll4/full/full/0/default.jpg

manifestUrl = ' https://digital.library.temple.edu/iiif/info/manifest.json'
responseManifest = requests.get(manifestUrl)
print(responseManifest.url)

# retrieving the metadata
m = json.loads(responseManifest.text)

#for i in m['collections']:
#      print(i['@id'])

In [ ]:
for r in m['collections']:
    print(r['@id'])
    print(r['label'])
    try:
        #for i in range(len(r['content']['descriptiveNonRepeating']['online_media']['media'])):
        for i in range(len(r['@id'])):
            idsId = str(r['@id']) - "https://cdm16002.contentdm.oclc.org/iiif/info/" 
            print(idsId)
            iiifUrl = r['@id']
            # manifestUrl = 'https://cdm16002.contentdm.oclc.org/iiif/info/p16002coll9/p1.json'
            iiifItemResponse = requests.get(iiifUrl)
            imageUrl = 'https://cdm16002.contentdm.oclc.org/iiif/image/' + idsID +'/full/full/0/default.jpg'
            print(imageUrl)
            iiifItem = json.loads(iiifItemResponse.text)
            title = date = licence = datasource = dimensions = sitter = typem = medium = artist =''
            for i in iiifItem['metadata']:
                if i['label'] == 'Title':
                    title = i['value']
                elif i['label'] == 'Date':
                    date = i['value']
                elif i['label'] == 'Contributor':
                    licence = i['value']
                elif i['label'] == 'Description':
                    datasource = i['value']
                elif i['label'] == 'Subject':
                    dimensions = i['value']
                else: pass
            csv_out.writerow([idsId,title,date,conributor,description,subject,iiifUrl,imageUrl])

    except:
        print("An exception occurred") 

# Query Temple Manifest Metadata (not working)

https://digital.library.temple.edu/iiif/info/p16002coll4/manifest.json

In [ ]:
# This appears to be the right way to query the Json:
#url = "https://cdm16002.contentdm.oclc.org/iiif/info/p16002coll4/4/manifest.json"
url = "https://cdm16002.contentdm.oclc.org/iiif/info/p16002coll9/2909/manifest.json"

In [ ]:
resp = urllib.request.urlopen(url)
data = resp.read().decode("utf-8")
data = json.loads(data)

In [ ]:
pid = 0
img_urls = {}
for c in data['sequences'][0]['canvases']:
    img_urls[pid]=c['images'][0]['resource']['service']['@id']
    print (img_urls[pid])
    pid = pid +1
print ("Ready.")

In [ ]:
df = pd.json_normalize(data)
df

In [ ]:
results = json.loads(data)

for r in results['response']['rows']:
    print(r['id'] + ' ' +  r['title'])
    print(r)
    # getting the identifiers of the records to access the IIIF manifests
    try:
        for i in range(len(r['content']['descriptiveNonRepeating']['online_media']['media'])):
            idsId = r['content']['descriptiveNonRepeating']['online_media']['media'][i]['idsId']
            print(idsId)
            iiifUrl = 'https://digital.library.temple.edu/iiif/ids/manifest/' + idsID
            iiifItemResponse = requests.get(iiifUrl)
            imageUrl = 'https://digital.library.temple.edu/iiif/' + idsID +'/full/full/0/default.jpg'
            print(imageUrl)
            iiifItem = json.loads(iiifItemResponse.text)
            title = date = licence = datasource = dimensions = sitter = typem = medium = artist =''
            for i in iiifItem['metadata']:
                if i['label'] == 'Title':
                    title = i['value']
                elif i['label'] == 'Date':
                    date = i['value']
                elif i['label'] == 'Creator (Person)':
                    licence = i['value']
                elif i['label'] == 'Subject':
                    datasource = i['value']
                elif i['label'] == 'Geographic Subject':
                    dimensions = i['value']
                elif i['label'] == 'Sitter':
                    sitter = i['value']
                elif i['label'] == 'Personal Names':
                    typem = i['value']
                elif i['label'] == 'Note':    
                    medium = i['value']
                elif i['label'] == 'Type':    
                    artist = i['value']
                else: pass
                
            csv_out.writerow([idsId,title,date,licence,datasource,dimensions,sitter,typem,medium,artist,iiifUrl,imageUrl])

    except:
        print("An exception occurred")  

In [ ]:
df = pd.read_csv('si_records.csv')

In [ ]:
len(df)

# Face Detection

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
test_image = cv2.imread('submission-example.jpg')

plt.imshow(test_image)

In [ ]:
print(test_image.shape)

In [ ]:
rgb_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
plt.imshow(rgb_image)

In [ ]:
#We define a function
def convertToRGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

## Load Classifier for Faces

In [ ]:
# Download from here and upload: https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalface_default.xml


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
haar_cascade_face = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
faces_rects = haar_cascade_face.detectMultiScale(test_image, scaleFactor = 1.2, minNeighbors = 5);

# Let us print the no. of faces found
print('Faces found: ', len(faces_rects))

In [ ]:
for (x,y,w,h) in faces_rects:
     cv2.rectangle(test_image, (x, y), (x+w, y+h), (0, 255, 0), 2)

In [ ]:
plt.imshow(convertToRGB(test_image))

# Download Portraits

In [ ]:
!mkdir images

for index, row in df.iterrows():
    print(index,     pid = 0
    img_urls = {}row['imageUrl'])
    response = requests.get(row['imageUrl'])
    img = Image.open(BytesIO(response.content))
    img.save('images/m-{}.jpg'.format(row['id']), quality=90)

## Process Images

In [ ]:
rows = 20
files = os.listdir('images/')

fig = plt.figure(figsize=(100,300))
for num, x in enumerate(files):
    #img = Image.open('is-images/'+ x)
    img = cv2.imread('images/'+ x)
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_rects = haar_cascade_face.detectMultiScale(img_gray, scaleFactor = 1.2, minNeighbors = 5);      
    
    for (x,y,w,h) in faces_rects:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
    #convert image to RGB and show image
    img_face = convertToRGB(img)    
    
    plt.subplot(rows,5,num+1)
    plt.axis('off')
    plt.imshow(img_face)

# ContentDM Python Wrapper

Source: https://github.com/saverkamp/pycdm

## Install Libraries

In [ ]:
#Unclear if pycdm is functional
#appears only available for Python 2.7
!pip install pycdm
import urllib3
from urllib3 import request
import certifi
import json
import pandas as pd

In [ ]:
http = urllib3.PoolManager(
    cert_reqs='CERT_REQUIRED',
    ca_certs=certifi.where())

## Querying ContentDM API





In [ ]:
url = 'https://server16002.contentdm.oclc.org/dmwebservices/index.php?q=dmGetCollectionList/json'

In [ ]:
r = http.request('GET', url)
r.status

In [ ]:
data = json.loads(r.data.decode('utf-8'))
data

In [ ]:
df = pd.json_normalize(data)
df